#  FrozenLake
Today you are going to learn how to survive walking over the (virtual) frozen lake through discrete optimization.

<img src="http://vignette2.wikia.nocookie.net/riseoftheguardians/images/4/4c/Jack's_little_sister_on_the_ice.jpg/revision/latest?cb=20141218030206" alt="a random image to attract attention" style="width: 400px;"/>


In [ ]:
# In google collab, uncomment this:
# !wget https://bit.ly/2FMJP5K -O setup.py && bash setup.py

# XVFB will be launched if you run on a server
# import os
# if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
#     !bash ../xvfb start
#     %env DISPLAY = : 1

In [1]:
import gym

#create a single game instance
env = gym.make("FrozenLake-v0")

#start new game
env.reset();

In [2]:
# display the game state
env.render()


SFFF
FHFH
FFFH
HFFG


### legend

![img](https://cdn-images-1.medium.com/max/800/1*MCjDzR-wfMMkS0rPqXSmKw.png)

### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [3]:
print("initial observation code:", env.reset())
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)

initial observation code: 0
printing observation:

SFFF
FHFH
FFFH
HFFG
observations: Discrete(16) n= 16
actions: Discrete(4) n= 4


In [4]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(2)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

taking action 2 (right)
new observation code: 4
reward: 0.0
is game over?: False
printing new state:
  (Right)
SFFF
FHFH
FFFH
HFFG


In [5]:
action_to_i = {
    'left':0,
    'down':1,
    'right':2,
    'up':3
}

### Play with it
* Try walking 5 steps without falling to the (H)ole
 * Bonus quest - get to the (G)oal
* Sometimes your actions will not be executed properly due to slipping over ice
* If you fall, call __env.reset()__ to restart

In [6]:
env.step(action_to_i['up'])
env.render()

  (Up)
SFFF
FHFH
FFFH
HFFG


### Policy

* The environment has a 4x4 grid of states (16 total), they are indexed from 0 to 15
* From each states there are 4 actions (left,down,right,up), indexed from 0 to 3

We need to define agent's policy of picking actions given states. Since we have only 16 disttinct states and 4 actions, we can just store the action for each state in an array.

This basically means that any array of 16 integers from 0 to 3 makes a policy.

In [7]:
import numpy as np
n_states = env.observation_space.n
n_actions = env.action_space.n

In [8]:
def get_random_policy():
    """
    Build a numpy array representing agent policy.
    This array must have one element per each of 16 environment states.
    Element must be an integer from 0 to 3, representing action
    to take from that state.
    """
    # policy = ...
    return policy

In [9]:
np.random.seed(1234)
policies = [get_random_policy() for i in range(10**4)]
assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should match n_actions-1'
action_probas = np.unique(policies, return_counts=True)[-1] /10**4. /n_states
print("Action frequencies over 10^4 samples:",action_probas)
assert np.allclose(action_probas, [1. / n_actions] * n_actions, atol=0.05), "The policies aren't uniformly random (maybe it's just an extremely bad luck)"
print("Seems fine!")

Action frequencies over 10^4 samples: [0.25014375 0.25130625 0.2495375  0.2490125 ]
Seems fine!


### Let's evaluate!
* Implement a simple function that runs one game and returns the total reward

In [10]:
def sample_reward(env, policy, t_max=100):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    Tip: see signature of env.step(...) method above.
    """
    s = env.reset()
    total_reward = 0
    
    for i in range(t_max):
        # action = ...
        s, reward, done, info = env.step(action)
        total_reward += reward
        if done:
            break
    return total_reward

In [11]:
print("generating 10^3 sessions...")
rewards = [sample_reward(env,get_random_policy()) for _ in range(10**3)]
assert all([type(r) in (int, float) for r in rewards]), 'sample_reward must return a single number'
assert all([0 <= r <= 1 for r in rewards]), 'total rewards should be between 0 and 1 for frozenlake (if solving taxi, delete this line)'
print("Looks good!")

generating 10^3 sessions...
Looks good!


In [12]:
def evaluate(policy, n_times=100):
    """Run several evaluations and average the score the policy gets."""
    # avg_reward = ...
    return avg_reward
        

In [13]:
def print_policy(policy):
    """a function that displays a policy in a human-readable way."""
    lake = "SFFFFHFHFFFHHFFG"
    assert env.spec.id == "FrozenLake-v0", "this function only works with frozenlake 4x4"
    
    # where to move from each tile
    arrows = ['<v>^'[a] for a in policy]
    
    #draw arrows above S and F only
    signs = [arrow if tile in "SF" else tile for arrow, tile in zip(arrows, lake)]
    
    for i in range(0, 16, 4):
        print(' '.join(signs[i:i+4]))

print("random policy:")
print_policy(get_random_policy())

random policy:
> < < >
v H > H
> ^ > H
H > < G


### Random search

In [19]:
best_policy = None
best_score = -float('inf')

from tqdm import tqdm
tr = tqdm(range(int(1e4)))
for i in tr:
    policy = get_random_policy()
    score = evaluate(policy)
    if score > best_score:
        best_score = score
        best_policy = policy
    tr.set_postfix({"best score:": best_score})

100%|██████████| 10000/10000 [03:18<00:00, 50.43it/s, best score:=0.71]


# Part II Genetic algorithm

The next task is to devise some more effecient way to perform policy search.
We'll do that with a bare-bones evolutionary algorithm.
[unless you're feeling masochistic and wish to do something entirely different which is bonus points if it works]

In [20]:
def crossover(policy1, policy2, p=0.5, prioritize=False):
    """
    for each state, with probability p take action from policy1, else policy2
    """
    if prioritize:
        # wait for part II - moar
        pass
    # policy = ...
    return policy


In [21]:
def mutation(policy, p=0.1):
    """
    for each state, with probability p replace action with random action
    Tip: mutation can be written as crossover with random policy
    """
    # new_policy = ...
    return new_policy

In [22]:
np.random.seed(1234)
policies = [
    crossover(get_random_policy(), get_random_policy()) 
    for i in range(10**4)]

assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should be n_actions-1'

assert any([
    np.mean(crossover(np.zeros(n_states), np.ones(n_states))) not in (0, 1)
    for _ in range(100)]), "Make sure your crossover changes each action independently"
print("Seems fine!")

Seems fine!


In [23]:

n_epochs = 20 #how many cycles to make
pool_size = 100 #how many policies to maintain
n_crossovers = 50 #how many crossovers to make on each step
n_mutations = 50 #how many mutations to make on each tick


In [24]:
print("initializing...")
pool = [get_random_policy() for _ in range(pool_size)]
pool_scores = list(map(evaluate, pool))


initializing...


In [25]:
assert type(pool) == type(pool_scores) == list
assert len(pool) == len(pool_scores) == pool_size
assert all([type(score) in (float, int) for score in pool_scores])

# Main loop

In [26]:
import random
from tqdm import tqdm

tr = tqdm(range(n_epochs))
for epoch in tr:
#     print("Epoch %s:"%epoch)
    crossovered = [
        crossover(random.choice(pool), random.choice(pool)) 
        for _ in range(n_crossovers)]
    mutated = [
        mutation(random.choice(pool)) 
        for _ in range(n_mutations)]
    
    assert type(crossovered) == type(mutated) == list
    
    # add new policies to the pool
    # pool = ...
    # pool_scores = ...
    
    # select pool_size best policies
    # selected_indices = ...
    # pool = ...
    # pool_scores = ...

    # print the best policy so far (last in ascending score order)
    tr.set_postfix({"best score:": pool_scores[-1]})

100%|██████████| 20/20 [00:58<00:00,  2.92s/it, best score:=0.79]


## moar

The parameters of the genetic algorithm aren't optimal, try to find something better. (size, crossovers and mutations)

Try alternative crossover and mutation strategies
* prioritize crossover for higher-scorers?
* try to select a more diverse pool, not just best scorers?
* Just tune the f*cking probabilities.

See which combination works best!

In [ ]:
def crossover(policy1, policy2, p=0.5, prioritize=False):
    """
    for each state, with probability p take action from policy1, else policy2
    """
    if prioritize:
        # the time has come
        pass
    # policy = ...
    return policy


In [27]:
import random
from tqdm import tqdm

tr = tqdm(range(n_epochs))
for epoch in tr:
#     print("Epoch %s:"%epoch)
    crossovered = [
        crossover(
            random.choice(pool), 
            random.choice(pool), 
            prioritize=True) 
        for _ in range(n_crossovers)]
    mutated = [
        mutation(random.choice(pool)) 
        for _ in range(n_mutations)]
    
    assert type(crossovered) == type(mutated) == list
    
    # add new policies to the pool
    # pool = ...
    # pool_scores = ...
    
    # select pool_size best policies
    # selected_indices = ...
    # pool = ...
    # pool_scores = ...

    # print the best policy so far (last in ascending score order)
    tr.set_postfix({"best score:": pool_scores[-1]})

100%|██████████| 20/20 [02:50<00:00,  8.52s/it, best score:=0.87]


# *** Part III

The frozenlake problem above is just too simple: you can beat it even with a random policy search. Go solve something more complicated. 

__FrozenLake8x8-v0__ - frozenlake big brother


### Some tips:
* Random policy search is worth trying as a sanity check, but in general you should expect the genetic algorithm (or anything you devised in it's place) to fare much better that random.
* While _it's okay to adapt the tabs above to your chosen env_, make sure you didn't hard-code any constants there (e.g. 16 states or 4 actions).
* `print_policy` function was built for the frozenlake-v0 env so it will break on any other env. You could simply ignore it OR write your own visualizer for bonus points.
* in function `sample_reward`, __make sure t_max steps is enough to solve the environment__ even if agent is sometimes acting suboptimally. To estimate that, run several sessions without time limit and measure their length.


# FrozenLake8x8

In [28]:
#create a single game instance
env = gym.make("FrozenLake8x8-v0")

#start new game
env.reset()

# display the game state
env.render()

n_states = env.observation_space.n
n_actions = env.action_space.n


SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


In [29]:
n_epochs = 20 #how many cycles to make
pool_size = 100 #how many policies to maintain
n_crossovers = 50 #how many crossovers to make on each step
n_mutations = 50 #how many mutations to make on each tick

In [30]:
print("initializing...")
pool = [get_random_policy() for _ in range(pool_size)]
pool_scores = list(map(evaluate, pool))

initializing...


In [31]:
import random
from tqdm import tqdm

tr = tqdm(range(n_epochs))
for epoch in tr:
#     print("Epoch %s:"%epoch)
    crossovered = [
        crossover(
            random.choice(pool), 
            random.choice(pool), 
            prioritize=True) 
        for _ in range(n_crossovers)]
    mutated = [
        mutation(random.choice(pool)) 
        for _ in range(n_mutations)]
    
    assert type(crossovered) == type(mutated) == list
    
    # add new policies to the pool
    # pool = ...
    # pool_scores = ...
    
    # select pool_size best policies
    # selected_indices = ...
    # pool = ...
    # pool_scores = ...

    # print the best policy so far (last in ascending score order)
    tr.set_postfix({"best score:": pool_scores[-1]})

100%|██████████| 20/20 [03:04<00:00,  9.20s/it, best score:=0.12]


# moar

In [32]:
def sample_reward(env, policy, t_max=200):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    Tip: see signature of env.step(...) method above.
    """
    s = env.reset()
    total_reward = 0
    
    for i in range(t_max):
        action = policy[s]
        s, reward, done, info = env.step(action)
        total_reward += reward
        if done:
            break
    return total_reward

In [33]:
# create a single game instance
env = gym.make("FrozenLake8x8-v0")

# start new game
env.reset()

# display the game state
env.render()

n_states = env.observation_space.n
n_actions = env.action_space.n


SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


In [34]:
n_epochs = 50 #how many cycles to make
pool_size = 200 #how many policies to maintain
n_crossovers = 100 #how many crossovers to make on each step
n_mutations = 100 #how many mutations to make on each tick

In [35]:
print("initializing...")
pool = [get_random_policy() for _ in range(pool_size)]
pool_scores = list(map(evaluate, pool))

initializing...


In [36]:
import random
from tqdm import tqdm

tr = tqdm(range(n_epochs))
for epoch in tr:
#     print("Epoch %s:"%epoch)
    crossovered = [
        crossover(
            random.choice(pool), 
            random.choice(pool), 
            prioritize=True) 
        for _ in range(n_crossovers)]
    mutated = [
        mutation(random.choice(pool)) 
        for _ in range(n_mutations)]
    
    assert type(crossovered) == type(mutated) == list
    
    # add new policies to the pool
    # pool = ...
    # pool_scores = ...
    
    # select pool_size best policies
    # selected_indices = ...
    # pool = ...
    # pool_scores = ...

    # print the best policy so far (last in ascending score order)
    tr.set_postfix({"best score:": pool_scores[-1]})

100%|██████████| 50/50 [24:24<00:00, 29.29s/it, best score:=0.94]
